# Evaluación práctica: Workshop de AI en Meteorología

<a target="_blank" href="https://colab.research.google.com/github/griverat/Meteo-AI/blob/main/notebooks/7.examen_practico.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Si usa Google Colab, asegúrese de tener habilitada la GPU para este notebook.**

![gpu_colab](https://github.com/griverat/Meteo-AI/blob/main/images/colab_gpu.png?raw=1)

El presente notebook consta de una serie de ejercicios que deberán ser completados por los participantes del Workshop de AI en Meteorología. El objetivo de estos ejercicios es evaluar los conocimientos adquiridos durante el desarrollo del workshop.

Para cada ejercicio se presentará una celda con un enunciado y, en la siguiente celda, un espacio para completar con el código necesario para resolver el ejercicio. Se pedirá que se complete el código faltante y que se ejecute la celda para verificar que el código funciona correctamente. Las secciones faltantes del código estarán marcadas con el comentario `# COMPLETAR` y estarán representadas por `__`.

Al finalizar el notebook, se deberá descargar el notebook con los ejercicios resueltos y subirlo al formulario de Google correspondiente.

¡Éxitos!


In [ ]:
!mkdir data
!wget https://github.com/griverat/Meteo-AI/raw/main/notebooks/data/temp_exam_data.nc -O data/temp_exam_data.nc
!pip install cartopy cmocean

In [ ]:
import cartopy.crs as ccrs
import cmocean as cmo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import xarray as xr

plt.rcParams["font.family"] = "monospace"


## Ejercicio 1

En este ejercicio se pide que se complete el código necesario para cargar un dataset de imágenes de satélite y se muestre una imagen de ejemplo.

### Enunciado

1. Cargar el dataset de temperatura desde la ruta `data/temp_exam_data.nc`.

2. Mostrar una imagen de ejemplo del dataset. Para ello, hacer uso de la función `sel` de `xarray` y seleccionar un paso de tiempo cualquiera. i.e. `ds.sel(time='2020-01-01')`. Recordar que `xarray` permite usar `.plot()` para visualizar los datos.

In [ ]:
# COMPLETAR
temp = xr.open_dataarray(____)
temp

In [ ]:
# COMPLETAR
temp.sel(____).____

Vamos a intentar pronosticar los valores de temperatura para la siguiente semana usando los datos de temperatura de la ultima semana, para ello prepararemos los datos en los grupos correspondientes.

In [ ]:
temp_input = temp.rolling(time=7).construct("channel").isel(time=slice(6, None))
temp_output = temp_input.copy(deep=True)

# ya que rolling asigna el tiempo al ultimo valor de la secuencia, lo corregimos restando 6 dias
# de esta manera el tiempo se asigna al valor inicial de la secuencia
temp_output["time"] = pd.to_datetime(temp_output.time) - pd.DateOffset(days=6)

# finalmente, desplazamos la secuencia en un paso hacia adelante
# de esta manera, el valor en el tiempo t, corresponde a los valores t+1, t+2, t+3, t+4, t+5, t+6
temp_output = temp_output.shift(time=-1).isel(time=slice(0, -1))

# seleccionamos los tiempos en comun
min_time = max(temp_input.time.min(), temp_output.time.min())
max_time = min(temp_input.time.max(), temp_output.time.max())

temp_input = temp_input.sel(time=slice(min_time, max_time))
temp_output = temp_output.sel(time=slice(min_time, max_time))

## Ejercicio 2

En este ejercicio se pide que se complete el código para separar el dataset en conjuntos de entrenamiento y validación.

### Enunciado

1. Separar el dataset en conjuntos de entrenamiento (90%) y pruebas (10%). No definiremos el conjunto de datos de validación en este caso.

2. Mostrar la cantidad de imágenes en cada conjunto.

In [ ]:
# COMPLETAR
train_size = int(__ * __)
test_size = len(temp) - train_size

# Impriimir los tamaños de los conjuntos de entrenamiento y prueba
print(___)
print(___)

In [ ]:
train_data = temp_input.isel(time=slice(0, train_size))
test_data = temp_input.isel(time=slice(train_size, None))

train_label = temp_output.isel(time=slice(0, train_size))
test_label = temp_output.isel(time=slice(train_size, None))

## Ejercicio 3

En este ejercicio se pide que se complete el código para modificar la cantidad de datos de entrada y salida (chnannels) del dataset.

### Enunciado

1. Modificar la cantidad de canales de entrada y salida del dataset. En este caso, se pide que se utilicen 7 canales de entrada y 7 canales de salida.

2. Inicializar un modelo de red neuronal convoluciona.

In [ ]:
def double_conv_block(x, n_filters):
    x = tf.keras.layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    x = tf.keras.layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    return x


def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = tf.keras.layers.MaxPool2D(2)(f)
    p = tf.keras.layers.Dropout(0.3)(p)
    return f, p


def upsample_block(x, conv_features, n_filters):
    x = tf.keras.layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    x = tf.keras.layers.concatenate([x, conv_features])
    x = tf.keras.layers.Dropout(0.3)(x)
    x = double_conv_block(x, n_filters)
    return x


def unet_res():
    # COMPLETAR
    inputs = tf.keras.layers.Input(shape=(16, 16, ___))

    f1, p1 = downsample_block(inputs, 64)
    f2, p2 = downsample_block(p1, 128)
    f3, p3 = downsample_block(p2, 256)
    f4, p4 = downsample_block(p3, 512)

    bottleneck = double_conv_block(p4, 1024)

    u6 = upsample_block(bottleneck, f4, 512)
    u7 = upsample_block(u6, f3, 256)
    u8 = upsample_block(u7, f2, 128)
    u9 = upsample_block(u8, f1, 64)

    # COMPLETAR
    outputs = tf.keras.layers.Conv2D(___, 1, padding="same", activation="linear")(u9)

    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model

In [ ]:
# COMPLETAR
unet_model = ___

## Ejercicio 4
En este ejercicio se pide que se complete el código para entrenar el modelo de red neuronal.

### Enunciado

1. Entrenar el modelo de red neuronal con los datos de entrenamiento. Escoger el optimizador de `adam` y la función de pérdida `mse`.

2. Escoger los hiperparámetros para el entrenamiento

In [ ]:
# COMPLETAR
unet_model.compile(optimizer=___, loss=___)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=10, monitor="val_loss", start_from_epoch=10
    ),
]

# COMPLETAR
history = unet_model.fit(
    train_data.data,
    train_label.data,
    epochs=___,
    batch_size=___,
    validation_split=0.1,
    callbacks=___,
)

## Ejercicio 5

En este ejercicio se pide que se complete el código para evaluar el modelo de red neuronal.

### Enunciado

1. Graficar la curva de aprendizaje del modelo.

2. Mostrar el error cuadrático medio obtenido. Recordar que las variables `test_data` y `test_label` contienen los datos de prueba y las etiquetas de prueba, respectivamente.

In [ ]:
# COMPLETAR
metrics = pd.DataFrame(___.history)
metrics.plot()

In [ ]:
# COMPLETAR
test_metrics = unet_model.evaluate(___, ___)
print(f"Test Loss: {test_metrics}")

## Evaluación del modelo

Al haber completado los ejercicios anteriores, el siguiente bloque de código graficara el resultado del modelo junto al resultado esperado y el error.

En esta sección no hay nada que completar, simplemente ejecutar el bloque de código.

In [ ]:
preds = unet_model.predict(test_data)
preds = xr.DataArray(
    preds,
    dims=["time", "lat", "lon", "channel"],
    coords={
        "time": test_label.time,
        "lat": test_label.lat,
        "lon": test_label.lon,
        "channel": temp_output.channel,
    },
)
preds

In [ ]:
# indice aleatorio
idx = np.random.randint(0, len(test_label))

fig, axs = plt.subplots(
    7, 3, figsize=(12, 28), subplot_kw={"projection": ccrs.PlateCarree()}
)

# en la primera columna se muestran los valores esperados
# en la segunda columna se muestran los valores predichos
# en la tercera columna se muestran las diferencias entre los valores esperados y predichos

plot_kwargs = dict(transform=ccrs.PlateCarree(), cmap=cmo.cm.thermal, vmin=-10, vmax=30)
date = pd.to_datetime(temp_output.time[idx].values)

for i, ax in enumerate(axs):
    for j, a in enumerate(ax):
        if j == 0:
            temp_output.isel(time=idx, channel=i).plot(ax=a, **plot_kwargs)
            a.set_title(f"True: {date+pd.DateOffset(days=i+1):%Y-%m-%d} (+{i+1} días)")
        elif j == 1:
            preds.isel(time=idx, channel=i).plot(ax=a, **plot_kwargs)
            a.set_title(f"Pred: {date+pd.DateOffset(days=i+1):%Y-%m-%d} (+{i+1} días)")
        else:
            (preds - temp_output).isel(time=idx, channel=i).plot(
                ax=a, transform=ccrs.PlateCarree(), cmap=cmo.cm.balance, vmin=-5, vmax=5
            )
            a.set_title(f"Diff: {date+pd.DateOffset(days=i+1):%Y-%m-%d} (+{i+1} días)")

        a.coastlines()

fig.suptitle(f"True vs Predicted - Inicialización: {date:%Y-%m-%d}", y=1)
fig.tight_layout()

## Opcional

Ahora que logramos entrenar un modelo de red neuronal para el pronóstico de temperatura en los siguientes 7 días, ¿cómo podríamos mejorar el modelo? ¿el modelo tiene información suficiente para realizar un pronóstico preciso? ¿el modelo, en su estado actual, es capaz de diferenciar entre estaciones del año?

### Respuesta
(Hacer doble click para editar)